In [1]:
import pandas as pd
#import requests 
#https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '04-monitoring/data/results-gpt4o-mini.csv'
docs_url = f'{base_url}/{relative_url}?raw=1'

data = pd.read_csv(docs_url)

In [2]:
data.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [3]:
df = data.iloc[:300]

In [4]:
del data

In [5]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [7]:
model_name = 'multi-qa-mpnet-base-dot-v1'
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

In [11]:
answer_llm = df.iloc[0].answer_llm

In [12]:
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [15]:
answer_llm_vector = embedding_model.encode(answer_llm)

### Q1

In [16]:
answer_llm_vector[0]

-0.42244673

In [17]:
import numpy as np

In [19]:
evaluations = []

for idx, row in df.iterrows():
    orig_v = embedding_model.encode(row['answer_orig'])
    llm_v = embedding_model.encode(row['answer_llm'])
    
    evaluations.append(llm_v.dot(orig_v))

In [24]:
evaluations[0]

17.515987

### Q2

In [23]:
np.percentile(evaluations,75)

31.674310207366943

In [27]:
def normalize(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm



In [28]:
evaluations = []

for idx, row in df.iterrows():
    orig_v = normalize(embedding_model.encode(row['answer_orig']))
    llm_v = normalize(embedding_model.encode(row['answer_llm']))        
    evaluations.append(llm_v.dot(orig_v))

### Q3

In [30]:
np.percentile(evaluations,75)

0.8362348675727844

In [31]:
!pip install rouge

In [32]:
from rouge import Rouge
rouge_scorer = Rouge()

In [34]:
r = df.iloc[10].to_dict()

In [35]:
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])

In [37]:
scores

[{'rouge-1': {'r': 0.45454545454545453,
   'p': 0.45454545454545453,
   'f': 0.45454544954545456},
  'rouge-2': {'r': 0.21621621621621623,
   'p': 0.21621621621621623,
   'f': 0.21621621121621637},
  'rouge-l': {'r': 0.3939393939393939,
   'p': 0.3939393939393939,
   'f': 0.393939388939394}}]

### Q4

In [48]:
scores[0]['rouge-1']['r']

0.45454545454545453

### Q5

In [47]:
(scores[0]['rouge-1']['r']+scores[0]['rouge-2']['r']+scores[0]['rouge-l']['r'])/3

0.35490035490035493

### Q6

In [54]:
evaluations = []

for idx, data in df.iterrows():
    scores = rouge_scorer.get_scores(data['answer_llm'], data['answer_orig'])
        
    rouge_1 = scores[0]['rouge-1']['f']
    rouge_2 = scores[0]['rouge-2']['f']
    rouge_l = scores[0]['rouge-l']['f']
    rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
    
    evaluations.append({'rouge_1': rouge_1,
                       'rouge_2': rouge_2,
                       'rouge_l': rouge_l,
                       'mean_rouge': rouge_avg})

In [59]:
for e in evaluations:
    print(e['rouge_2'])

0.028169010918468917
0.05555555225694465
0.17777777313333343
0.047058819111419105
0.03389830142092829
0.2580645130072841
0.17948717488494426
0.1403508750384734
0.3243243199415632
0.21428570946712028
0.21621621121621637
0.42105262658241
0.6075949317288897
0.5714285664994332
0.4878048730963712
0.2666666616671202
0.28235293642629766
0.11904761433106596
0.06521738638941436
0.22222221734320996
0.08247422278669378
0.13483145731599552
0.3275862021105827
0.07499999781250007
0.27027026561155754
0.3749999950086806
0.18181817759412314
0.22018348159245865
0.43548386603147765
0.21153845707285512
0.20833332847222233
0.2553191440470802
0.13953487874526788
0.10389610005059889
0.0
0.15384615003287322
0.10256409875082198
0.28571428143990935
0.09523809096371902
0.5624999950439454
0.3243243193279767
0.20588234801038074
0.3768115892543584
0.18918918419284161
0.08163264957934209
0.0
0.0
0.0
0.0
0.0
0.214285709286235
0.2164948403629505
0.08648648151117633
0.2553191447532821
0.14201182946675553
0.505494500524

In [60]:
[(lambda x: x['rouge_2']) for i in evaluations]

[<function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function __main__.<listcomp>.<lambda>(x)>,
 <function